# Paso 0 - Instalar dependencias

Antes de empezar a trabajar con AWS Lambda, necesitamos configurar una cuenta de AWS

```bash

$ mkdir ~/.aws
$ cat >> ~/.aws/credentials
[default]
aws_access_key_id=YOUR_ACCESS_KEY_HERE
aws_secret_access_key=YOUR_SECRET_ACCESS_KEY
region=YOUR_REGION (such as us-west-2, us-west-1, etc)
```


In [ ]:
client.create_alias(
    FunctionName='LambdaExample1',
    Name='DEV',
    FunctionVersion='$LATEST',
    Description='DEV alias of LambdaExample1'
)

# Paso 1 - Ejemplos Integración

## 1.1 - Ejemplo básico usando boto3

Usando la librería de boto3 podemos desplegar una función simple con este script:

In [1]:
import boto3


client = boto3.client("lambda")

client.create_function(
    FunctionName='ejemplo_lambda_simple',
    Role='arn:aws:iam::016081714405:role/service-role/LambdaTest',
    Description='Ejemplo básico de AWS Lambda',
    Runtime='python3.6',
    Handler='myexamplelambdaproject.lambda1.main.my_handler',
    Code={
        'S3Bucket': 'lambdartefacts',
        'S3Key': 'myexamplelambdaproject.zip',
    },
)

{'CodeSha256': 'm3Spw2Db5E6taeL+gqlCUvv4JlLs3jPiPewvCtkFbjQ=',
 'CodeSize': 1635,
 'Description': 'Ejemplo básico de AWS Lambda',
 'FunctionArn': 'arn:aws:lambda:eu-west-1:016081714405:function:ejemplo_lambda_simple',
 'FunctionName': 'ejemplo_lambda_simple',
 'Handler': 'myexamplelambdaproject.lambda1.main.my_handler',
 'LastModified': '2017-07-31T06:36:39.314+0000',
 'MemorySize': 128,
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '640',
   'content-type': 'application/json',
   'date': 'Mon, 31 Jul 2017 06:36:39 GMT',
   'x-amzn-requestid': '9b80b688-75ba-11e7-8397-fda84bb378f7'},
  'HTTPStatusCode': 201,
  'RequestId': '9b80b688-75ba-11e7-8397-fda84bb378f7',
  'RetryAttempts': 0},
 'Role': 'arn:aws:iam::016081714405:role/service-role/LambdaTest',
 'Runtime': 'python3.6',
 'Timeout': 3,
 'TracingConfig': {'Mode': 'PassThrough'},
 'Version': '$LATEST'}

## 1.2 - Alias

La única forma para trabajar con versiones y alias en AWS Lambda, por el momento, es a través de la API de AWS.
Crearemos una versión suponiendo un alias para cada entorno de desarrollo

In [4]:
client.create_alias(
    FunctionName='ejemplo_lambda_simple',
    Name='Testing',
    FunctionVersion='$LATEST',
    Description='DEV alias of ejemplo_lambda_simple'
)

{'AliasArn': 'arn:aws:lambda:eu-west-1:016081714405:function:ejemplo_lambda_simple:Testing',
 'Description': 'DEV alias of ejemplo_lambda_simple',
 'FunctionVersion': '$LATEST',
 'Name': 'Testing',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '187',
   'content-type': 'application/json',
   'date': 'Mon, 31 Jul 2017 06:37:15 GMT',
   'x-amzn-requestid': 'b12ea359-75ba-11e7-b0f8-cf7e2ee4d438'},
  'HTTPStatusCode': 201,
  'RequestId': 'b12ea359-75ba-11e7-b0f8-cf7e2ee4d438',
  'RetryAttempts': 0}}

Verificamos que todo se ha creado correctamente desde la consola de AWS o con este método:

In [ ]:
client.list_aliases(
    FunctionName='ejemplo_lambda_simple',
)


## 1.3 - Version

Creamos una nueva versión y la asociamos a nuestros Alias:

In [ ]:
client.publish_version(
    FunctionName='ejemplo_lambda_simple',
)
client.update_alias(
    FunctionName='ejemplo_lambda_simple',
    Name='DEV',
    FunctionVersion='1', # Versión a la que queremos asociarla
    Description='PRE alias of ejemplo_lambda_simple'
) 

## 1.4 - Triggers

Los triggers son los eventos que hacen que se ejecuten las funciones de AWS Lambda. Por desgracia, la API de AWS Lambda no tiene implementado la creación de los eventos. Para poder crearlos de manera programática, nos tendremos que apoyar en las APIS de otros servicios como S3, Cloudwatch, SNS...

En este caso, nos apoyaremos en otra librería para poder desplegar estos eventos.

**importante:** Si manejamos los eventos por API, estos no aparecerán en la pestaña de "Triggers" de la consola de AWS.

In [3]:
%%bash
cd 1.4-triggers/
ardy -p myexamplelambdaproject deploy

1.1-simple
1.3-triggers
2.1-testing
examples.ipynb
README.rst
requirements.txt
venv


## 1.5 - Api Gateway

Ver en consola


# 2 - Ejemplos CI y Testing

![Jenkins 1](statics/img/jenkins01.png "Jenkins 1")

## 2.1 Virtual Machine

Para llevar nuestra Integración Continua, utilizaremos Jenkins con el módulo de pipeline.

Para crear un entorno lo más parecido a producción y trabajar con Jenkins, utilizaremos [Vagrant](https://www.vagrantup.com) para crear una máquina virtual y [Ansible](https://www.ansible.com) para provisionar la VM.



In [1]:

%%bash

cd 2.1-testing
# vagrant up
# vagrant ssh

Bringing machine 'default' up with 'virtualbox' provider...
==> default: Importing base box 'centos/7'...
==> default: Matching MAC address for NAT networking...
==> default: Checking if box 'centos/7' is up to date...
==> default: Setting the name of the VM: 21-testing_default_1499973967796_7811
==> default: Clearing any previously set network interfaces...
==> default: Preparing network interfaces based on configuration...
    default: Adapter 1: nat
    default: Adapter 2: hostonly
==> default: Forwarding ports...
    default: 22 (guest) => 2222 (host) (adapter 1)
==> default: Booting VM...
==> default: Waiting for machine to boot. This may take a few minutes...
    default: SSH address: 127.0.0.1:2222
    default: SSH username: vagrant
    default: SSH auth method: private key
    default: 
    default: Vagrant insecure key detected. Vagrant will automatically replace
    default: this with a newly generated keypair for better security.
    default: 
    default: Inserting generate

## 2.2 Jenkins y CI

- Podremos acceder a Jenkins en:
http://192.168.100.3:8080/

- Plugins necesarios:
https://wiki.jenkins.io/display/JENKINS/HTML+Publisher+Plugin

- Crear credenciales:
http://192.168.100.3:8080/credentials/
![Jenkins 3](statics/img/jenkins03.png "Jenkins 3")


- Crear proyecto: http://192.168.100.3:8080/view/all/newJob
![Jenkins 2](statics/img/jenkins02.png "Jenkins 2")


- Añadir este script en el Pipeline de proyecto:


```groovy

node{
    stage("Commit Stage") {
        git([url: "https://github.com/avara1986/aws-lambda-python-ci.git", branch: "master"])
    }
    stage("Build Stage") {
        withCredentials([usernamePassword(credentialsId: "aws", passwordVariable: "AWS_SECRET_ACCESS_KEY", usernameVariable: "AWS_ACCESS_KEY_ID")]) {
            sh "aws configure set aws_access_key_id ${AWS_ACCESS_KEY_ID}"
            sh "aws configure set aws_secret_access_key ${AWS_SECRET_ACCESS_KEY}"
            sh "aws configure set region eu-west-1"
            sh "aws configure set output json"
        }
        env.WORKSPACE = pwd()
        sh "virtualenv --python=/usr/local/bin/python3.6 venv"
        env.PATH="${env.WORKSPACE}/venv/bin:/usr/bin:${env.PATH}"
        sh "pip install -r requirements-test.txt"
    }
    stage("Test Stage") {
        sh script: "./tests.sh"
    }
    stage("Deploy Stage") {
        withCredentials([usernamePassword(credentialsId: "aws", passwordVariable: "AWS_SECRET_ACCESS_KEY", usernameVariable: "AWS_ACCESS_KEY_ID")]) {
            sh "ardy -p myexamplelambdaproject deploy"
        }
        
    }
    stage("Coverage&amp;Sonar Stage") {
        publishHTML([allowMissing: false, alwaysLinkToLastBuild: false, keepAll: false, reportDir: "htmlcov", reportFiles: "index.html", reportName: "HTML Report"])
    }
}
```